In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
try:
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
except ImportError:
    from keras.preprocessing.image import ImageDataGenerator
    from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [3]:
TRAIN_DIR = r'c:\Users\Abhay Tyagi\OneDrive - ABES\Desktop\4th Year Project\Datasets\Brain Tumor Dataset\Training'
TEST_DIR = r'c:\Users\Abhay Tyagi\OneDrive - ABES\Desktop\4th Year Project\Datasets\Brain Tumor Dataset\Testing'
print("Training directory:", TRAIN_DIR)
print("Testing directory:", TEST_DIR)

Training directory: c:\Users\Abhay Tyagi\OneDrive - ABES\Desktop\4th Year Project\Datasets\Brain Tumor Dataset\Training
Testing directory: c:\Users\Abhay Tyagi\OneDrive - ABES\Desktop\4th Year Project\Datasets\Brain Tumor Dataset\Testing


In [4]:
print("Training dir exists:", os.path.exists(TRAIN_DIR))
print("Testing dir exists:", os.path.exists(TEST_DIR))

Training dir exists: True
Testing dir exists: True


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=False,
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)
# Load test data
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 974 images belonging to 2 classes.
Found 243 images belonging to 2 classes.
Found 220 images belonging to 2 classes.


In [6]:
print("Classes found:", train_generator.class_indices)
print("Number of training samples:", train_generator.samples)
print("Number of validation samples:", validation_generator.samples)
print("Number of test samples:", test_generator.samples)

Classes found: {'meningioma_tumor': 0, 'no_tumor': 1}
Number of training samples: 974
Number of validation samples: 243
Number of test samples: 220


In [7]:
def create_brain_tumor_model():
    # Create input layer
    inputs = tf.keras.Input(shape=(224, 224, 3))
    
    # Create ResNet50 base model
    base_model = tf.keras.applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Freeze base model
    base_model.trainable = False
    
    # Build model using Functional API (more stable for saving/loading)
    x = base_model(inputs, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(128, activation='relu', name='dense_1')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(2, activation='softmax', name='predictions')(x)
    
    model = tf.keras.Model(inputs, outputs)
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create and compile the model
model = create_brain_tumor_model()

# Build the model with a dummy input to initialize weights
dummy_input = tf.random.normal((1, 224, 224, 3))
_ = model(dummy_input)

print("Model created and compiled successfully!")
print("\nModel Architecture:")
model.summary()

# Check trainable parameters
total_params = model.count_params()
trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
print(f"\nTotal Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Non-trainable Parameters: {total_params - trainable_params:,}")

Model created and compiled successfully!

Model Architecture:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ predictions (Dense)             │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,850,242 (90.98 MB)

 Trainable params: 262,530 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)


Total Parameters: 23,850,242
Trainable Parameters: 262,530
Non-trainable Parameters: 23,587,712


In [8]:
# Add this cell to check class distribution
import os

def check_dataset_quality():
    train_dir = r'c:\Users\Abhay Tyagi\OneDrive - ABES\Desktop\4th Year Project\Datasets\Brain Tumor Dataset\Training'
    
    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"{class_name}: {count} images")
    
    print("\nClass balance check:")
    print("Ideal: Each class should have similar number of images")
    print("Problem: If one class has <50% of others, it's imbalanced")

check_dataset_quality()

meningioma_tumor: 822 images
no_tumor: 395 images

Class balance check:
Ideal: Each class should have similar number of images
Problem: If one class has <50% of others, it's imbalanced


In [9]:
import shutil
import time
from datetime import datetime

# Define callbacks for better training
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

print("Starting training...")
print(f"Trainable parameters before training: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}")

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Create directory and save model with unique name
try:
    # Ensure directory exists
    os.makedirs('../trained_models', exist_ok=True)
    
    # Create unique filename to avoid conflicts
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f'../trained_models/brain_tumor_model_{timestamp}.h5'
    
    # Save the model
    model.save(model_filename)
    print(f"Model saved successfully as: {model_filename}")
    
    # Also save with standard name (overwrite if exists)
    try:
        model.save('../trained_models/brain_tumor_model.h5')
        print("Model also saved as: brain_tumor_model.h5")
    except Exception as e:
        print(f"Could not save with standard name: {e}")
        print("But timestamped version was saved successfully!")
        
except Exception as e:
    print(f"Error saving model: {e}")

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Safe plotting with fallback
try:
    import matplotlib
    matplotlib.use('Agg')  # Non-interactive backend
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    
    # Ensure directory exists for plots
    os.makedirs('../trained_models', exist_ok=True)
    plt.savefig('../trained_models/training_plots.png', dpi=300, bbox_inches='tight')
    plt.close()  # Close figure to free memory
    print("Training plots saved successfully!")
    
except Exception as e:
    print(f"Could not create plots: {e}")

print("Training completed!")

# Print training summary
print("\n=== Training Summary ===")
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Models saved in: ../trained_models/")

Starting training...
Trainable parameters before training: 262,530
Epoch 1/30


c:\Users\Abhay Tyagi\OneDrive - ABES\Desktop\4th Year Project\Project\Medical_Image_Analysis_System\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


30/30 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 0.6133 - loss: 0.8612 - val_accuracy: 0.6830 - val_loss: 0.6236 - learning_rate: 0.0010
Epoch 2/30
 1/30 ━━━━━━━━━━━━━━━━━━━━ 28s 995ms/step - accuracy: 0.8125 - loss: 0.4895

C:\Users\Abhay Tyagi\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.8125 - loss: 0.2529 - val_accuracy: 0.8421 - val_loss: 0.1931 - learning_rate: 0.0010
Epoch 3/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.6598 - loss: 0.6589 - val_accuracy: 0.7500 - val_loss: 0.5227 - learning_rate: 0.0010
Epoch 4/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6562 - loss: 0.4452 - val_accuracy: 0.7368 - val_loss: 0.2616 - learning_rate: 0.0010
Epoch 5/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.7172 - loss: 0.6010 - val_accuracy: 0.7366 - val_loss: 0.5223 - learning_rate: 0.0010
Epoch 6/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.6250 - loss: 0.3361 - val_accuracy: 0.6316 - val_loss: 0.3185 - learning_rate: 0.0010
Epoch 7/30
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7309 - loss: 0.5589
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
30/30 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.7300 - loss: 0.5599 - val_accuracy: 0.7455 - v

Error saving model: Unable to synchronously create dataset (name already exists)
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.6545 - loss: 0.6769
Test Accuracy: 0.5682
Training plots saved successfully!
Training completed!

=== Training Summary ===
Final Training Accuracy: 0.6562
Final Validation Accuracy: 0.8421
Test Accuracy: 0.5682
Models saved in: ../trained_models/


In [10]:
# Convert existing model to native Keras format (Keras 3 recommended)
try:
    # Save in native Keras format (.keras extension)
    model.save('../trained_models/brain_tumor_model.keras')
    print("✅ Model saved in native Keras format successfully!")
except Exception as e:
    print(f"❌ Keras format conversion failed: {e}")
    
    # Fallback: Try TensorFlow's SavedModel format
    try:
        import tensorflow as tf
        tf.saved_model.save(model, '../trained_models/brain_tumor_savedmodel')
        print("✅ Model saved as TensorFlow SavedModel successfully!")
    except Exception as e2:
        print(f"❌ TensorFlow SavedModel also failed: {e2}")

✅ Model saved in native Keras format successfully!
